![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Agentic tasks with AutoGen and Redis

This notebook demonstrates how to build an agent using Microsoft's [AutoGen](https://microsoft.github.io/autogen/stable//index.html) agent framework and the RedisMemory integration.

We'll define an agent, give it access to tools and memory, then set in on a task to see how it uses its abilities.

## Let's Begin!

<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/agents/04_autogen_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Install packages and set up Redis

In [ ]:
%pip install -q autogen autogen_agentchat transformers
%pip install -q "autogen-ext[redisvl]"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### For Colab download and run a Redis instance

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

'\n%%sh\ncurl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg\necho "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list\nsudo apt-get update  > /dev/null 2>&1\nsudo apt-get install redis-stack-server  > /dev/null 2>&1\nredis-stack-server --daemonize yes\n'

In [3]:
import json
import os
import re
import requests
from collections import Counter
from transformers import pipeline
from typing import List

/Users/justin.cechmanek/.pyenv/versions/3.11.9/envs/redis-ai-res/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


## Building our agent

We'll be building a product review writing agent that takes in a set of product descriptions, collects relevant information, and provides a summary and analysis you can use for SEO.


### Defining tools
One of the defining the features of an agent is its ability to use tools so let's give it some.

For use with Autogen that just requires we define a well named function with type hints in its signature

In [5]:

async def summarize(restaurant_name: str, all_reviews: List[str]) -> str:
    """takes a list of reviews for a single restaurant and returns a summary of all of them."""
    # set up a summarizer model
    summarizer = pipeline('summarization', model='facebook/bart-large-cnn')
    # pass all the reviews
    summary = summarizer('\n'.join(all_reviews),  # concatenate all the reviews together
                max_length=1024,
                min_length=128,
                do_sample=False)[0]["summary_text"]
    return restaurant_name + ": " + summary


async def get_keywords(full_text: str) -> List[str]:
    """extract the most commonly occurring keywords present in the reviews to know
    which terms it is likely to rank highly for in keyword search engines."""
    # define a set of common English stopwords to ignore
    STOPWORDS = {
        'the', 'of', 'and', 'to', 'for', 'in', 'on', 'at', 'a', 'an', 'is', 'it', 'its', 'with', 'as', 'by', 'from', 'that',
        'this', 'those', 'be', 'are', 'was', 'were', 'or', 'but', 'not', 'so', 'if', 'then', 'than', 'which', 'who', 'whom',
        'about', 'into', 'out', 'up', 'down', 'over', 'under', 'again', 'further', 'once', 'here', 'there', 'when',
        'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no',
        'nor', 'only', 'own', 'same', 'can', 'will', 'just', 'don', 'should', 'now', 'has', 'have', 'had', 'do', 'does',
        'did', 'their', 'them', 'they', 'you', 'your', 'yours', 'he', 'him', 'his', 'she', 'her', 'hers', 'we', 'us',
        'our', 'ours', 'i', 's', 'me', 'my', 'mine', 'also', 'place'
    }
    # remove punctuation and lowercase the text
    words = re.findall(r'\b\w+\b', full_text.lower())
    # filter out stopwords
    filtered_words = [word for word in words if word not in STOPWORDS]
    # count occurrences
    word_counts = Counter(filtered_words)
    # return the top 10
    return [word for word, _ in word_counts.most_common(10)]


async def publish_article(final_draft: str, file_name:str= "food_article.md") -> str:
    "accepts the final version of an article, writes it to a markdown file and returns the full file location path."
    with open(file_name, 'w') as file:
        file.write(final_draft)

    full_path = os.path.abspath(__file__)
    return os.path.join(full_path, file_name)

## Adding relevant memories
Our agent needs to know what people think of these restaurants so we'll add the user reviews to our agent memory powered by Redis.


In [6]:
# fetch the reviews from our public S3 bucket
# the original dataset can be found here: https://www.kaggle.com/datasets/jkgatt/restaurant-data-with-100-trip-advisor-reviews-each
def fetch_data(file_name):
    dataset_path = 'datasets/'
    try:
        with open(dataset_path + file_name, 'r') as f:
            return json.load(f)
    except:
        url = 'https://redis-ai-resources.s3.us-east-2.amazonaws.com/recommenders/datasets/two-towers/'
        r = requests.get(url + file_name)
        if not os.path.exists(dataset_path):
            os.makedirs(dataset_path)
        with open(dataset_path + file_name, 'wb') as f:
            f.write(r.content)
        return json.loads(r.content.decode('utf-8'))

In [7]:
restaurant_data = fetch_data('factual_tripadvisor_restaurant_data_all_100_reviews.json')

print(f"we have {restaurant_data['restaurant_count']} restaurants in our dataset, with {restaurant_data['total_review_count']} total reviews")

restaurant_reviews = restaurant_data["restaurants"] # ignore the count fields

# drop some of the fields that we don't need
for restaurant in restaurant_reviews:
    for field in ['region', 'country', 'tel', 'fax', 'email', 'website', 'address_extended', 'chain_name', 'trip_advisor_url']:
        if field in restaurant:
            restaurant.pop(field)

we have 147 restaurants in our dataset, with 14700 total reviews


In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.memory.redis import RedisMemory, RedisMemoryConfig
from autogen_ext.models.openai import OpenAIChatCompletionClient
from logging import WARNING, getLogger

logger = getLogger()
logger.setLevel(WARNING)

# initailize Redis memory
redis_memory = RedisMemory(
    config=RedisMemoryConfig(
        redis_url="redis://localhost:6379",
        index_name="restaurant_reviews",
        prefix="trip_advisor",
    )
)

for restaurant in restaurant_reviews:
    # add each review to our agent memory
    # for brevity we'll take only the first 10 reviews per restaurant
    for review in restaurant['reviews'][:10]:
        try:
            await redis_memory.add(
                MemoryContent(
                    content= " ".join([review.pop("review_title"),
                                       review.pop("review_text"),
                                       str({key: val for key, val in restaurant.items() if key != "reviews"})]),
                    mime_type=MemoryMimeType.TEXT,
                    metadata={},
                )
            )
        except KeyError:
            pass

## Create our agent and set it on a task

In [9]:

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Create assistant agent with ChromaDB memory
review_agent = AssistantAgent(
    name="restaurant_review_agent",
    system_message="you are an experienced writer and food critic. ",
    model_client=model_client,
    tools=[summarize , get_keywords, publish_article],
    memory=[redis_memory],
    max_tool_iterations=5,
)

writing_task = "Write an article reviewing the restaurants in the San Francisco bay area. \
            Include a brief summary of the most popular restaurants based on the user reviews. \
            Group them into categories based on their cuisine and price, and talk about the \
            top rated restaurants in each category."

stream = review_agent.run_stream(task=writing_task)
await Console(stream)

---------- TextMessage (user) ----------
Write an article reviewing the restaurants in the San Francisco bay area.             Include a brief summary of the most popular restaurants based on the user reviews.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.
---------- MemoryQueryEvent (restaurant_review_agent) ----------
[MemoryContent(content="Good food, very busy Food is very good and good drinks. We booked table in advance which is recommended as the place is very busy.The service is attentive and prompt (almost too prompt). Not much time between the meals. The tables are very close, so not much privacy.We had a table located close to the door.. Every time someone entered or left the restaurant we felt the cold draft from outside. Since the restaurant is very busy it continued all the time, which was actually quite uncomfortable. The rating is based on the food. {'name': 'Colibri Mexican 

Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 111. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 193. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 77. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 242. Since this is a summarization task, where outputs shorter than the input are typicall

---------- ToolCallExecutionEvent (restaurant_review_agent) ----------
[FunctionExecutionResult(content="Colibri Mexican Bistro: Service is attentive and prompt (almost too prompt). Not much time between the meals. The tables are very close, so not much privacy. We had a table located close to the door. Every time someone entered or left the restaurant we felt the cold draft from outside. Since the restaurant is very busy it continued all the time, which was actually quite uncomfortable. Good food, very busy. Food is very good and good drinks. We booked table in advance which is recommended as the place isvery busy. The rating is based on the food. The restaurant is located in the heart of London's West End. It is open from 7am to 10pm.", name='summarize', call_id='call_qiKuu59aMG3sICgcBznOdjYu', is_error=False), FunctionExecutionResult(content="Lori's Diner: The food was good: as you would expect better than average fast food which tasted fresh and homemade. The service was poor: we w

TaskResult(messages=[TextMessage(id='679fc2fa-89b1-4f75-92ec-c45b354ef1e9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 18, 49, 54, 550970, tzinfo=datetime.timezone.utc), content='Write an article reviewing the restaurants in the San Francisco bay area.             Include a brief summary of the most popular restaurants based on the user reviews.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.', type='TextMessage'), MemoryQueryEvent(id='596b3ee5-d5c0-4740-81e8-2c6855e14458', source='restaurant_review_agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 18, 49, 54, 660417, tzinfo=datetime.timezone.utc), content=[MemoryContent(content="Good food, very busy Food is very good and good drinks. We booked table in advance which is recommended as the place is very busy.The service is attentive and prompt (almost too prompt). Not

## Follow up tasks
Our agent doesn't have to be a one-and-done worker. We can ask it to continue toward our overall goal of having a well viewed food critic article.

You've probably noticed the agent's output is somewhat messy. That's ok as our final article will be written cleanly to a markdown file.

In [10]:
from autogen_agentchat.messages import TextMessage

task_list = [TextMessage(source='user', content="Now analyze your article and tell me the key search terms it is likely to rank highly for."),
             TextMessage(source='user', content="Using your analysis suggest changes to the original article to improve keyword ranking."),
             TextMessage(source='user', content="Based on your suggestions, edit and modify your article to improve SEO keyword ranking. Give a new list of top keywords"),
             TextMessage(source='user', content="When it is ready, publish the article by saving it to a markdown file.")
]
stream =  review_agent.run_stream(task=task_list)
await Console(stream)

---------- TextMessage (user) ----------
Now analyze your article and tell me the key search terms it is likely to rank highly for.
---------- TextMessage (user) ----------
Using your analysis suggest changes to the original article to improve keyword ranking.
---------- TextMessage (user) ----------
Based on your suggestions, edit and modify your article to improve SEO keyword ranking. Give a new list of top keywords
---------- TextMessage (user) ----------
When it is ready, publish the article by saving it to a markdown file.
---------- ToolCallRequestEvent (restaurant_review_agent) ----------
[FunctionCall(id='call_rABbThm3fmTWqiYGw0F3LZYR', arguments='{"full_text":"Culinary Adventures in the San Francisco Bay Area: A Comprehensive Guide\\n\\nThe San Francisco Bay Area is a haven for food enthusiasts, offering a dynamic and vibrant culinary scene. In this article, we take you on a journey through some of the most popular restaurants in the area, gathering insights from user reviews.

TaskResult(messages=[TextMessage(id='e5bd757a-e3f2-494e-bb47-a966802d956f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 18, 51, 1, 302724, tzinfo=datetime.timezone.utc), content='Now analyze your article and tell me the key search terms it is likely to rank highly for.', type='TextMessage'), TextMessage(id='ad745802-6c0e-47e2-979d-715f8f1af217', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 18, 51, 1, 302800, tzinfo=datetime.timezone.utc), content='Using your analysis suggest changes to the original article to improve keyword ranking.', type='TextMessage'), TextMessage(id='dbe6f357-22b0-4eb5-a075-8bb12078e4cd', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 18, 51, 1, 302810, tzinfo=datetime.timezone.utc), content='Based on your suggestions, edit and modify your article to improve SEO keyword ranking. Give a new list of top keywords', type='TextMessage'), TextM

### Clean up
close out our agent and empty our Redis index

In [11]:
await model_client.close()
await redis_memory.close()